# Home Credit Default Risk

Can you predict how capable each applicant is of repaying a loan?
Many people struggle to get loans due to insufficient or non-existent credit histories. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the unbanked population by providing a positive and safe borrowing experience. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

In [9]:
# Import some libraries needed for data analysis
import numpy as np
import pandas as pd
from IPython.display import display # Allows the use of display() for DataFrames

# Set random seed
SEED = 42

# Pretty display for notebooks
%matplotlib inline

# Load the Application Train dataset
df_application_train = pd.read_csv("data/application_train.csv")

# Display the first record
df_application_train.head(n=1)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
# Split the data into features and target label
target = df_application_train['TARGET']
features = df_application_train.drop('TARGET', axis = 1)
features.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
target.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [11]:
# Create new columns from flags
features['OWN_CAR'] = features['FLAG_OWN_CAR'].apply(lambda x: 1 if x == 'Y' else 0)
features['OWN_REALTY'] = features['FLAG_OWN_REALTY'].apply(lambda x: 1 if x == 'Y' else 0)

In [12]:
# One-hot encode some categorical features using pandas.get_dummies()
dummy_cols = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE']

# Create dummy
dummy = pd.get_dummies(features[dummy_cols])

# Concat dummy
features_final = pd.concat([features, dummy], axis=1)

# drop original columns
features_final.drop(dummy_cols, axis=1, inplace=True)

In [13]:
# Drop remaining columns that will not be used for benchmark model
drop_cols = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
            'NAME_TYPE_SUITE', 'NAME_FAMILY_STATUS', 'DAYS_BIRTH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
             'FLAG_WORK_PHONE','FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS',
            'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
             'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
             'REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
             'EXT_SOURCE_3', 'APARTMENTS_AVG','BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
             'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG',
             'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG',
             'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
             'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
             'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 
             'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
             'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 
             'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 
             'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 
             'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 
             'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
             'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
             'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
             'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10',
             'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
             'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
             'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR' ,'AMT_REQ_CREDIT_BUREAU_DAY',
             'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 
             'AMT_REQ_CREDIT_BUREAU_YEAR']

features_final.drop(drop_cols, axis=1, inplace=True)

In [14]:
features_final

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,REGION_RATING_CLIENT,...,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,100002,202500.000,406597.5,24700.5,351000.0,0.018801,-637,-3648.0,-2120,2,...,0,0,0,0,0,0,0,0,0,0
1,100003,270000.000,1293502.5,35698.5,1129500.0,0.003541,-1188,-1186.0,-291,1,...,0,0,0,0,0,0,0,0,0,0
2,100004,67500.000,135000.0,6750.0,135000.0,0.010032,-225,-4260.0,-2531,2,...,0,0,0,0,0,0,0,0,0,0
3,100006,135000.000,312682.5,29686.5,297000.0,0.008019,-3039,-9833.0,-2437,2,...,0,0,0,0,0,0,0,0,0,0
4,100007,121500.000,513000.0,21865.5,513000.0,0.028663,-3038,-4311.0,-3458,2,...,0,0,0,0,0,0,0,0,0,0
5,100008,99000.000,490495.5,27517.5,454500.0,0.035792,-1588,-4970.0,-477,2,...,0,0,0,0,0,0,0,0,0,0
6,100009,171000.000,1560726.0,41301.0,1395000.0,0.035792,-3130,-1213.0,-619,2,...,0,0,0,0,0,0,0,0,0,0
7,100010,360000.000,1530000.0,42075.0,1530000.0,0.003122,-449,-4597.0,-2379,3,...,0,0,0,0,0,0,0,0,0,0
8,100011,112500.000,1019610.0,33826.5,913500.0,0.018634,365243,-7427.0,-3514,2,...,0,0,0,0,0,0,0,0,0,1
9,100012,135000.000,405000.0,20250.0,405000.0,0.019689,-2019,-14437.0,-3992,2,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Replace some missing values with median
features_final['AMT_ANNUITY'].fillna((features_final['AMT_ANNUITY'].median()), inplace=True)
features_final['AMT_GOODS_PRICE'].fillna((features_final['AMT_GOODS_PRICE'].median()), inplace=True)

In [20]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the features and target data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, target, test_size=0.2, random_state=SEED)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 246008 samples.
Testing set has 61503 samples.


In [38]:
print(y_train.value_counts())

0    226132
1     19876
Name: TARGET, dtype: int64


In [21]:
# Check if any columns have missing values
features_final.columns[features_final.isnull().any()].tolist()

[]

In [22]:
# Display the first record again
display(features_final.head(n=1))

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,REGION_RATING_CLIENT,...,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,100002,202500.0,406597.5,24700.5,351000.0,0.018801,-637,-3648.0,-2120,2,...,0,0,0,0,0,0,0,0,0,0


In [23]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
print(clf.feature_importances_)

C:\Users\ntdtr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.         0.         0.09881495 0.         0.13289729 0.00477427
 0.09766606 0.08130337 0.0544089  0.04404948 0.05251679 0.
 0.         0.         0.         0.         0.10358952 0.
 0.         0.         0.15611653 0.         0.07956313 0.
 0.         0.01216359 0.         0.         0.         0.
 0.         0.02107287 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.004622   0.
 0.01163544 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00024484 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

In [31]:
y_pred = clf.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.10f}'.format(clf.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.9195323805


In [32]:
from sklearn.metrics import recall_score
recall_score(y_pred, y_test)

C:\Users\ntdtr\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


0.0

In [33]:
from sklearn.metrics import precision_score
precision_score(y_pred, y_test)

0.0

In [28]:

# Import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# Create classifier
dt_clf = DecisionTreeClassifier(max_depth=10, max_features='sqrt', random_state=SEED)

# Fit model
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [29]:
print(dt_clf.score(X_test, y_test))

0.9191258962977416


In [ ]:
import visuals as vs
import seaborn as sns
# Set Seaborn color codes
sns.set(color_codes=True)

vs.plot_roc_auc_curve(y_test, y_scores[:, 1])